In [3]:
%matplotlib inline
import boto3
import botocore
import os.path
import xarray as xr
import datetime

import numpy as np
import pandas as pd
import xarray as xr
from dateutil.relativedelta import relativedelta
import pickle
from matplotlib import animation, pyplot as plt
from pathlib import Path
import calendar
import os

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from imblearn.over_sampling import SMOTE

import seaborn as sns

In [4]:
# Specify your longitude and latitude range
lats = slice(25, 0)
lons = slice(70, 80)

# Initialize a list to store the data
data_list = []

# Iterate over the years and months
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-5-31')

curr_date = start_date
while curr_date <= end_date:
    year = curr_date.year
    month = curr_date.month
    
    # Get the number of days in the month
    _, num_days = calendar.monthrange(year, month)
    
    # Construct the file path
    file_path = f'/home/jovyan/shared/data_sst/{year}{month:02d}_sea_surface_temperature.nc'
    
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}. Skipping this month.')
        curr_date += relativedelta(months=1)
        continue
    
    # Open the dataset
    ds = xr.open_dataset(file_path)
    
    # Select the time points
    time_points = pd.date_range(start=f'{year}-{month:02d}-01T09:00:00', end=f'{year}-{month:02d}-{num_days}T09:00:00', freq='D')
    ds_selected_time = ds.sel(time0=time_points, method='nearest')
    
    # Select the longitude and latitude
    ds_selected = ds_selected_time.sel(lon=lons, lat=lats)
    
    # Get the sea_surface_temperature values
    sst_data = ds_selected['sea_surface_temperature'].values
    sst_data = np.expand_dims(sst_data, axis=-1)

    # Append the data to the list
    data_list.append(sst_data)
    
    # Move to the next month
    curr_date += relativedelta(months=1)
    
    # Print a message
    print(f'Successfully processed data for: {year}-{month:02d}')

# Concatenate all the data
sst_data = np.concatenate(data_list, axis=0)

print(sst_data.shape)

Successfully processed data for: 2020-01
File not found: /home/jovyan/shared/data_sst/202002_sea_surface_temperature.nc. Skipping this month.
Successfully processed data for: 2020-03
Successfully processed data for: 2020-04
Successfully processed data for: 2020-05
Successfully processed data for: 2020-06
Successfully processed data for: 2020-07
Successfully processed data for: 2020-08
Successfully processed data for: 2020-09
Successfully processed data for: 2020-10
Successfully processed data for: 2020-11
Successfully processed data for: 2020-12
Successfully processed data for: 2021-01
Successfully processed data for: 2021-02
Successfully processed data for: 2021-03
Successfully processed data for: 2021-04
Successfully processed data for: 2021-05
Successfully processed data for: 2021-06
Successfully processed data for: 2021-07
Successfully processed data for: 2021-08
Successfully processed data for: 2021-09
Successfully processed data for: 2021-10
Successfully processed data for: 2021-

In [5]:
processed_data_list = []

for day_data in sst_data:
    day_data = np.squeeze(day_data)
    
    day_data = np.flipud(day_data)
    
    sst_data2 = np.nanmean(day_data)
    
    sst_data3 = day_data - sst_data2
    
    processed_data_list.append(sst_data3)

processed_data = np.array(processed_data_list)

In [6]:
processed_data_reshaped = processed_data.reshape((processed_data.shape[0], -1))
SST_DATA = np.nan_to_num(processed_data_reshaped, nan=0.0)

In [29]:
start_year = 2020
end_year = 2023

# List to store all group labels
all_labels = pd.Series(dtype=int)

# Process each year
for year in range(start_year, end_year + 1):
    # Process each month
    for month in range(1, 13):
        if year == 2023 and month > 5:
            break
        file_name = f"/home/jovyan/shared/SST_Diff/{year}{month:02d}_sst_diff.nc"
        if not Path(file_name).exists():
            print(f"File {file_name} does not exist, skipping...")
            continue
        print(f"Processing file {file_name}...")

        # Open the dataset and convert it to a pandas DataFrame
        temp = xr.open_dataset(file_name)
        temp_df = temp.to_dataframe()

        # Convert the 'time' column to date only format
        temp_df.index = pd.to_datetime(temp_df.index).date

        # Reset index to create 'time' column
        temp_df = temp_df.reset_index()

        # Create a new column 'entry_id' which enumerates each entry within each date
        temp_df['entry_id'] = temp_df.groupby('time').cumcount()

        # Filter dataframe for rows where 'entry_id' is 22 and 23, and create a copy
        filtered_df = temp_df[temp_df['entry_id'].isin([22,23])].copy()

        # Create a label column - set to 1 if SST_Diff < threshold, else set to 0
        threshold = -1
        filtered_df.loc[:, 'label'] = np.where(filtered_df['SST_Diff'] < threshold, 1, 0)

        # Group by 'time' and check if all 'label' in a group is 1, if so, set group_label to 1, else 0
        group_labels = filtered_df.groupby('time')['label'].apply(lambda x: 1 if x.sum() == 2 else 0)

        # Append the group_labels to all_labels
        all_labels = pd.concat([all_labels, group_labels])
print(all_labels.shape)

Processing file /home/jovyan/shared/SST_Diff/202001_sst_diff.nc...
File /home/jovyan/shared/SST_Diff/202002_sst_diff.nc does not exist, skipping...
Processing file /home/jovyan/shared/SST_Diff/202003_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202004_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202005_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202006_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202007_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202008_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202009_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202010_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202011_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202012_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202101_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202102_sst_diff.nc...
Processing file /home/jovyan/shared/SST_Diff/202

In [30]:
all_labels_reshaped = all_labels.values.ravel()

In [31]:
print("Shape of sst_data_reshaped: ", SST_DATA.shape)
print("Shape of all_labels_reshaped: ", all_labels_reshaped.shape)

Shape of sst_data_reshaped:  (1218, 4141)
Shape of all_labels_reshaped:  (1218,)


In [47]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Masking
from keras.regularizers import l1_l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from tqdm.keras import TqdmCallback
import numpy as np

# Reshape data for SMOTE
sst_data_flat = SST_DATA.reshape(SST_DATA.shape[0], -1)

# Apply SMOTE
smote = SMOTE(random_state=42)
sst_data_res, labels_res = smote.fit_resample(sst_data_flat, all_labels_reshaped)

# Reshape the data to original shape
sst_data_res = sst_data_res.reshape((sst_data_res.shape[0], -1, 1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sst_data_res, labels_res, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(X_train.shape[1], 1)))
model.add(LSTM(150, activation='tanh', return_sequences=True, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.2))
model.add(LSTM(100, activation='tanh', return_sequences=True, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='tanh', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Compute class weights
classes = np.unique(y_train)
class_weights = class_weight.compute_sample_weight('balanced', y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

tqdm_callback = TqdmCallback(verbose=1)

# Train the model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=0, callbacks=[early_stopping, tqdm_callback], class_weight=class_weight_dict)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

14/14 [==============================] - 26s 2s/step - loss: 0.3753 - accuracy: 0.8367
Accuracy: 0.8366890549659729


In [38]:
# Start date
start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2023-5-31')
excluded_months = ['198304', '198802', '199209', '199702', '200111', '200604', '201102', '201506', '202002']

# Initialize an empty list to store the rows of the DataFrame
data = []

# Initialize the current index of sst_data and all_labels_reshaped
curr_index = 0

# Iterate over the dates
curr_date = start_date
while curr_date <= end_date:
    year_month = curr_date.strftime('%Y%m')
    if year_month in excluded_months:
        _, num_days = calendar.monthrange(curr_date.year, curr_date.month)
        curr_index += num_days
        curr_date += relativedelta(months=1)
        continue
    _, num_days = calendar.monthrange(curr_date.year, curr_date.month)
    for _ in range(num_days):
        # Check if the current index is out of bounds
        if curr_index >= len(all_labels_reshaped):
            break
        # Get the actual value
        actual_val = all_labels_reshaped[curr_index]

        # Get the prediction
        day_data = SST_DATA[curr_index].reshape(1, -1, 1)
        prediction = model.predict(day_data)

        # Append the data to the list
        data.append({'time': curr_date, 'actual_val': actual_val, 'predicted_val': prediction})

        # Increase the current index and the current date
        curr_index += 1
        curr_date += pd.DateOffset(days=1)

    # Exit the loop if the current index is out of bounds
    if curr_index >= len(all_labels_reshaped):
        break

# Create the DataFrame
df = pd.DataFrame(data)

# Check the DataFrame
print(df)

1/1 [==============================] - 1s 667ms/step
           time  actual_val      predicted_val
0    2020-01-01           0   [[7.973488e-05]]
1    2020-01-02           0   [[7.980182e-05]]
2    2020-01-03           0   [[7.979543e-05]]
3    2020-01-04           0  [[7.9794896e-05]]
4    2020-01-05           0   [[7.977359e-05]]
...         ...         ...                ...
1184 2023-04-28           0  [[0.00041262517]]
1185 2023-04-29           0   [[0.0014725414]]
1186 2023-04-30           0    [[0.004187423]]
1187 2023-05-01           0   [[0.0027750377]]
1188 2023-05-02           0   [[0.0019705466]]

[1189 rows x 3 columns]


In [54]:
print(len(df[(df['actual_val'] == 0) & (df['predicted_val'] > 0.77)]))
print(len(df[(df['actual_val'] == 0)]))
print(len(df[(df['actual_val'] == 1) & (df['predicted_val'] > 0.77)]))
print(len(df[(df['actual_val'] == 1)]))

335
1088
100
101


In [55]:
print(len(df[(df['actual_val'] == 0) & (df['predicted_val'] > 0.5)]))
print(len(df[(df['actual_val'] == 0)]))
print(len(df[(df['actual_val'] == 1) & (df['predicted_val'] > 0.5)]))
print(len(df[(df['actual_val'] == 1)]))

375
1088
101
101


In [56]:
model.save('LSTM(Stopped)_202001-202305.h5')

NameError: name 'all_labels' is not defined